In [17]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split 
from sklearn.datasets import make_blobs 
from sklearn import svm

from h import *
from W_unitary import *
from U_unitary import *
from qiskit.algorithms.optimizers import COBYLA, ADAM, SPSA, SLSQP, POWELL, L_BFGS_B, TNC, AQGD

%matplotlib inline
#plt.rcParams['figure.figsize'] = (10,6)
#plt.rcParams['figure.dpi'] = 100
#sns.set()

#%matplotlib inline
#%load_ext autoreload
#%autoreload 2

In [18]:
def MSE_loss(theta, X_train, y_train):
    
    # new_y_train = []
    # for i in range(0, len(y_train)):
    #     if y_train[i] == 0:
    #         new_y_train.append(-1)
    #     else:
    #         new_y_train.append(1)
    
    predictions = []
    for i in range(0, len(y_train)):
        predictions.append(h(X_train[i], theta, 2, 2, shots))
    
    error = []
    for i in range(0, len(predictions)):
        parity = predictions[i] - y_train[i]
        error.append(parity)

    norm = np.linalg.norm(error)
    return norm


def cross_validation_split(X,y, ele_per_split, i):
    X_train =  np.concatenate((X[:ele_per_split*i,:], X[ ele_per_split*(i+1) :,:]))
    X_test = X[ele_per_split*i:ele_per_split*(i+1),:]
    y_train = np.concatenate((y[ :ele_per_split*i], y[ele_per_split*(i+1):]))
    y_test = y[ ele_per_split*i:ele_per_split*(i+1)]
    return X_train, X_test, 2*y_train-1, 2*y_test-1

In [19]:


RANDOM_STATE=42
seed = np.random.seed(RANDOM_STATE)
theta = 2*np.pi*np.random.random(2*2*3)

n_samples = 100
n_splits = 4

X, y = make_moons(n_samples, noise=0.8)
if n_samples%n_splits != 0:
    print("Preffered to have n_splits is divisor of n_samples")
ele_per_split = n_samples//n_splits

shots = 1024

In [20]:
thetas = np.zeros((len(theta), n_splits))
for i in range(n_splits):
    X_train, X_test, y_train, y_test = cross_validation_split(X,y, ele_per_split, i)

    objective_function = lambda theta: MSE_loss(theta, X_train, y_train)

    optimizer = COBYLA(maxiter=100)
    theta_opt = optimizer.minimize(objective_function, theta).x

    thetas[:,i] = theta_opt

    y_optimizer_train = np.zeros(len(y_train))
    for j in range(0, len(y_train)):
        y_optimizer_train[j] = (h(X_train[j], theta_opt, 2, 2, shots))

    y_optimizer_test = np.zeros(len(y_test))
    for j in range(0, len(y_test)):
        y_optimizer_test[j] = (h(X_test[j], theta_opt, 2, 2, shots))

    print("Split ", i)
    print("Optimal Theta ", theta_opt)
    print("Performance on training set: ",int(1/2*np.sum(abs(y_optimizer_train-y_train))), "incorrectly out of ", n_samples - n_samples//n_splits)
    print("Performance on test set ",1/2*np.sum(abs(y_optimizer_test-y_test)), "incorrectly out of ", n_samples//n_splits)

Split  0
Optimal Theta  [ 2.13821293  5.61911007  4.52680389  3.6696238   0.41825407  2.10106424
  0.05000139  5.31818031  3.73553155  3.87481028 -0.26696127  5.8663242 ]
Performance on training set:  30 incorrectly out of  75
Performance on test set  12.0 incorrectly out of  25
Split  1
Optimal Theta  [ 2.32139736  6.64065236  4.59925358  3.60040333  0.98599066  2.10862556
  0.12122374  5.44234523  3.1226292   4.44895122 -0.69081281  5.96564025]
Performance on training set:  32 incorrectly out of  75
Performance on test set  12.0 incorrectly out of  25
Split  2
Optimal Theta  [ 2.154275    5.95595357  3.36601705  4.06884343  0.61139801  1.2318776
  0.17165753  5.24750894  3.5820309   6.04918313 -0.07336508  6.08902337]
Performance on training set:  28 incorrectly out of  75
Performance on test set  14.0 incorrectly out of  25
Split  3
Optimal Theta  [ 3.62051049  5.04362981  4.58346227  3.44202831  0.94665115  0.84169301
  0.54234994  5.40342646  5.23984427  5.87442635 -1.31714589  5.

In [21]:
theta_opt_avr = np.mean(thetas, axis=1)
X_val, y_val = make_moons(n_samples//n_splits, noise=0.8)

y_val = 2*y_val-1
y_optimizer_val = np.zeros(len(y_val))
for j in range(0, len(y_val)):
    y_optimizer_val[j] = (h(X_val[j], theta_opt, 2, 2, shots))

print("Average Theta ", theta_opt_avr)
print("Performance on validation set: ",1/2*np.sum(abs(y_optimizer_val-y_val)), "incorrectly out of ", n_samples//n_splits)
print(sum(y_val!=y_optimizer_val))
# print(y_val)
# print(y_optimizer_val)
# print(y_val==y_optimizer_val)

Average Theta  [ 2.55859895  5.81483645  4.2688842   3.69522472  0.74057347  1.5708151
  0.22130815  5.35286524  3.92000898  5.06184274 -0.58707126  5.89740901]
Performance on validation set:  17.0 incorrectly out of  25
17
